In [1]:
import math
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.distributions
from collections import namedtuple
from itertools import count

device = "cpu"
floattype = torch.float

batchsize = 512
nsamples = 8
npoints = 5
emsize = 512


class Graph_Transformer(nn.Module):
    def __init__(self, emsize = 32, nhead = 8, nhid = 1024, nlayers = 3, ndecoderlayers = 0, dropout = 0):
        super().__init__()
        self.emsize = emsize
        from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer
        encoder_layers = TransformerEncoderLayer(emsize, nhead, nhid, dropout = dropout)
        decoder_layers = TransformerDecoderLayer(emsize, nhead, nhid, dropout = dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.transformer_decoder = TransformerDecoder(decoder_layers, ndecoderlayers)
        self.encoder = nn.Linear(3, emsize)
        self.outputattention_query = nn.Linear(emsize, emsize, bias = False)
        self.outputattention_key = nn.Linear(emsize, emsize, bias = False)
        self.start_token = nn.Parameter(torch.randn([emsize], device = device))
    
    def generate_subsequent_mask(self, sz): #last dimension will be softmaxed over when adding to attention logits, if boolean the ones turn into -inf
        #mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        #mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        
        mask = torch.triu(torch.ones([sz, sz], dtype = torch.bool, device = device), diagonal = 1)
        return mask
    
    def encode(self, src): #src must be [batchsize * nsamples, npoints, 3]
        src = self.encoder(src).transpose(0, 1)
        output = self.transformer_encoder(src)
        return output #[npoints, batchsize * nsamples, emsize]
    
    def decode_next(self, memory, tgt, route_mask): #route mask is [batchsize * nsamples, npoints], both memory and tgt must have batchsize and nsamples in same dimension (the 1th one)
        npoints = memory.size(0)
        batchsize = tgt.size(1)
        """if I really wanted this to be efficient I'd only recompute the decoder for the last tgt, and just remebering what the others looked like from before (won't change due to mask)"""
        """have the option to freeze the autograd on all but the last part of tgt, although at the moment this is a very natural way to say: initial choices matter more"""
        tgt_mask = self.generate_subsequent_mask(tgt.size(0))
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[tgt, batchsize * nsamples, emsize]
        output_query = self.outputattention_query(memory).transpose(0, 1) #[batchsize * nsamples, npoints, emsize]
        output_key = self.outputattention_key(output[-1]) #[batchsize * nsamples, emsize]
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.unsqueeze(-1)).squeeze(-1) #[batchsize * nsamples, npoints], technically don't need to scale attention as we divide by variance next anyway
        output_attention_tanh = output_attention.tanh() #[batchsize * nsamples, npoints]
        
        #we clone the route_mask incase we want to backprop using it (else it was modified by inplace opporations)
        output_attention = output_attention.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        output_attention_tanh = output_attention_tanh.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        
        return output_attention_tanh, output_attention #[batchsize * nsamples, npoints]
    
    def calculate_logprob(self, memory, routes): #memory is [npoints, batchsize * nsamples, emsize], routes is [batchsize * nsamples, npoints - 4], rather than backproping the entire loop, this saves vram (and computation)
        npoints = memory.size(0)
        ninternalpoints = routes.size(1)
        bigbatchsize = memory.size(1)
        memory_ = memory.gather(0, routes.transpose(0, 1).unsqueeze(2).expand(-1, -1, self.emsize)) #[npoints - 4, batchsize * nsamples, emsize] reorder memory into order of routes
        tgt = torch.cat([self.start_token.unsqueeze(0).unsqueeze(1).expand(1, bigbatchsize, -1), memory_[:-1]]) #[npoints - 4, batchsize * nroutes, emsize], want to go from memory to tgt
        tgt_mask = self.generate_subsequent_mask(ninternalpoints)
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[npoints - 4, batchsize * nsamples, emsize]
        """want probability of going from key to query, but first need to normalise (softmax with mask)"""
        output_query = self.outputattention_query(memory_).transpose(0, 1) #[batchsize * nsamples, npoints - 4, emsize]
        output_key = self.outputattention_key(output).transpose(0, 1) #[batchsize * nsamples, npoints - 4, emsize]
        attention_mask = torch.full([ninternalpoints, ninternalpoints], True, device = device).triu(1) #[npoints - 4, npoints - 4], True for i < j
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.transpose(-1, -2))
        """quick fix to stop divergence"""
        output_attention_tanh = output_attention.tanh()
        
        output_attention_tanh = output_attention_tanh.masked_fill(attention_mask, float('-inf'))
        output_attention_tanh = output_attention_tanh - output_attention_tanh.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 4, npoints - 4]
        
        output_attention = output_attention.masked_fill(attention_mask, float('-inf'))
        output_attention = output_attention - output_attention.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 4, npoints - 4]
        
        """infact I'm almost tempted to not mask choosing a previous point, so it's forced to learn it and somehow incorporate it into its computation, but without much impact on reinforcing good examples"""
        logprob_tanh = output_attention_tanh.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        logprob = output_attention.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        return logprob_tanh, logprob #[batchsize * nsamples]

NN = Graph_Transformer().to(device)
optimizer = optim.Adam(NN.parameters())


class environment:    
    def reset(self, npoints, batchsize, nsamples=1, corner_points = None, initial_triangulation = None):
        """
        corner_points, etc., shoudn't include a batch dimension
        """
        if corner_points == None:
            ncornerpoints = 4
        else:
            ncornerpoints = corner_points.size(0)
        if npoints <= ncornerpoints:
            print("Error: not enough points for valid problem instance")
            return
        self.batchsize = (
            batchsize * nsamples
        )  # so that I don't have to rewrite all this code, we store these two dimensions together
        self.nsamples = nsamples
        self.npoints = npoints
        self.points = (
            torch.rand([batchsize, npoints - ncornerpoints, 3], dtype = floattype, device=device)
            .unsqueeze(1)
            .expand(-1, nsamples, -1, -1)
            .reshape(self.batchsize, npoints - ncornerpoints, 3)
        )
        if corner_points == None:
            self.corner_points = torch.tensor(
                [[0, 0, 0], [3, 0, 0], [0, 3, 0], [0, 0, 3]], dtype = floattype, device=device
            )
        else:
            self.corner_points = corner_points
        self.points = torch.cat(
            [
                self.corner_points.unsqueeze(0).expand(self.batchsize, -1, -1),
                self.points,
            ],
            dim=-2,
        )  # [batchsize * nsamples, npoints, 3]
        self.points_mask = torch.cat(
            [
                torch.ones([self.batchsize, ncornerpoints], dtype=torch.bool, device=device),
                torch.zeros(
                    [self.batchsize, npoints - ncornerpoints], dtype=torch.bool, device=device
                ),
            ],
            dim=1,
        )
        self.points_sequence = torch.empty(
            [self.batchsize, 0], dtype=torch.long, device=device
        )

        """
        points are now triples
        triangles are now quadruples
        edges are now still just indices, but there are four of them per 'triangle', and they correspond to triples of points, not pairs
        we use  0,2,1  0,3,2  0,1,3  1,2,3  as the order of the four 'edges'/faces
        opposite face is always ordered such that the last two indices are swapped
        faces are always read ANTICLOCKWISE
        
        first three points of tetrahedron MUST be read clockwise (from the outside) to get correct sign on incircle test
        
        new point will be inserted in zeroth position, so if corresponding face of REMOVED tetrahedron is [x,y,z] (being read anticlockwise from outside in) new tetrahedron is [p, x, y, z]
        """
        
        """
        number of tetrahedra is not the same for each batch (in 3D), so store as a big list, and remember batch index that it comes from
        """
        if corner_points == None:
            initial_triangulation = torch.tensor([[0, 1, 2, 3]], dtype=torch.long, device=device)
        
        self.partial_delaunay_triangles = initial_triangulation.unsqueeze(0).expand(self.batchsize, -1, -1).reshape(-1, 4)
        self.batch_index = torch.arange(self.batchsize, dtype = torch.long, device = device).unsqueeze(1).expand(-1, initial_triangulation.size(0)).reshape(-1)
        
        self.batch_triangles = self.partial_delaunay_triangles.size(0) #[0]
        self.ntriangles = torch.full([self.batchsize], initial_triangulation.size(0), dtype = torch.long, device = device) #[self.batchsize]
        
        self.cost = torch.zeros([self.batchsize], dtype = floattype, device=device)

        self.logprob = torch.zeros([self.batchsize], dtype = floattype, device=device, requires_grad=True)

    def update(self, point_index):  # point_index is [batchsize]
        
        assert point_index.size(0) == self.batchsize
        assert str(point_index.device) == device
        assert self.points_mask.gather(1, point_index.unsqueeze(1)).sum() == 0
        
        triangles_coordinates = self.points[self.batch_index.unsqueeze(1), self.partial_delaunay_triangles] # [batch_triangles, 4, 3]
        
        newpoint = self.points[self.batch_index, point_index[self.batch_index]] # [batch_triangles, 3]

        incircle_matrix = torch.cat(
            [
                newpoint.unsqueeze(1),
                triangles_coordinates,
            ],
            dim=-2,
        )  # [batch_triangles, 5, 3]
        incircle_matrix = torch.cat(
            [
                (incircle_matrix * incircle_matrix).sum(-1, keepdim=True),
                incircle_matrix,
                torch.ones([self.batch_triangles, 5, 1], dtype = floattype, device=device),
            ],
            dim=-1,
        )  # [batch_triangles, 5, 5]
        assert incircle_matrix.dtype == floattype
        assert str(incircle_matrix.device) == device
        
        incircle_test = (
            incircle_matrix.det() > 0
        )  # [batch_triangles], is True if inside incircle
        
        conflicts = incircle_test.sum()
        
        conflicting_triangles = self.partial_delaunay_triangles[incircle_test] # [conflicts, 4]
        
        conflicting_edges_index0 = torch.empty_like(conflicting_triangles)
        indices = torch.LongTensor([0, 0, 0, 1])
        conflicting_edges_index0 = conflicting_triangles[:, indices] # [conflicts, 4]
        
        conflicting_edges_index1 = torch.empty_like(conflicting_triangles)
        indices = torch.LongTensor([2, 3, 1, 2])
        conflicting_edges_index1 = conflicting_triangles[:, indices] # [conflicts, 4]
        
        conflicting_edges_index2 = torch.empty_like(conflicting_triangles)
        indices = torch.LongTensor([1, 2, 3, 3])
        conflicting_edges_index2 = conflicting_triangles[:, indices] # [conflicts, 4]
        
        conflicting_edges = torch.cat([conflicting_edges_index0.view(-1).unsqueeze(-1), conflicting_edges_index1.view(-1).unsqueeze(-1), conflicting_edges_index2.view(-1).unsqueeze(-1)], dim = -1).reshape(-1, 3) # [conflicts * 4, 3]
        
        edge_batch_index = self.batch_index[incircle_test].unsqueeze(1).expand(-1, 4).reshape(-1) # [conflicts * 4]
        
        indices = torch.LongTensor([0, 2, 1])
        comparison_edges = conflicting_edges[:, indices] # [conflicts * 4, 3]        
        
        unravel_nomatch_mask = torch.ones([conflicts * 4], dtype = torch.bool, device = device) # [conflicts * 4]
        i = 1
        while True:
            
            todo_mask = unravel_nomatch_mask[:-i].logical_and(edge_batch_index[:-i] == edge_batch_index[i:])
            if i % 4 == 0:
                if todo_mask.sum() == 0:
                    break
            
            match_mask = todo_mask.clone()
            match_mask[todo_mask] = (conflicting_edges[:-i][todo_mask] != comparison_edges[i:][todo_mask]).sum(-1).logical_not()
            
            unravel_nomatch_mask[:-i][match_mask] = False
            unravel_nomatch_mask[i:][match_mask] = False
            
            i += 1
        
        batch_newtriangles = unravel_nomatch_mask.sum()
        
        nomatch_edges = conflicting_edges[unravel_nomatch_mask] # [batch_newtriangles, 3], already in correct order to insert into 1,2,3 (since already anticlockwise from outside in)
        assert list(nomatch_edges.size()) == [batch_newtriangles, 3]
        nomatch_batch_index = edge_batch_index[unravel_nomatch_mask] # [batch_newtriangles]
        
        nomatch_newpoint = point_index[nomatch_batch_index] # [batch_newtriangles]
        
        newtriangles = torch.cat([nomatch_newpoint.unsqueeze(1), nomatch_edges], dim = -1) # [batch_newtriangles, 4]
        
        
        nremoved_triangles = torch.zeros([self.batchsize], dtype = torch.long, device = device)
        nnew_triangles = torch.zeros([self.batchsize], dtype = torch.long, device = device)
        
        indices = self.batch_index[incircle_test]
        nremoved_triangles.put_(indices, torch.ones_like(indices, dtype = torch.long), accumulate = True) # [batchsize]
        
        indices = edge_batch_index[unravel_nomatch_mask]
        nnew_triangles.put_(indices, torch.ones_like(indices, dtype = torch.long), accumulate = True) # [batchsize]
        
        assert (nnew_triangles <= 2 * nremoved_triangles + 2).logical_not().sum().logical_not()
        
        """
        NOTE:
        I THINK it's possible for nnew_triangles to be less than nremoved_triangles (or my code is just buggy...)
        """
        
        assert nnew_triangles.sum() == batch_newtriangles
        assert nremoved_triangles.sum() == incircle_test.sum()
        
        nadditional_triangles = nnew_triangles - nremoved_triangles # [batchsize]
        ntriangles = self.ntriangles + nadditional_triangles # [batchsize]
        
        partial_delaunay_triangles = torch.empty([ntriangles.sum(), 4], dtype = torch.long, device = device)
        batch_index = torch.empty([ntriangles.sum()], dtype = torch.long, device = device)
        
        cumulative_triangles = torch.cat([torch.zeros([1], dtype = torch.long, device = device), nnew_triangles.cumsum(0)[:-1]]) # [batchsize], cumulative sum starts at zero
        
        """
        since may actually have LESS triangles than previous round, we insert all that survive into the first slots (in that batch)
        """
        good_triangle_indices = torch.arange(incircle_test.logical_not().sum(), dtype = torch.long, device = device)
        good_triangle_indices += cumulative_triangles[self.batch_index[incircle_test.logical_not()]]
        bad_triangle_indices_mask = torch.ones([ntriangles.sum(0)], dtype = torch.bool, device = device)
        bad_triangle_indices_mask.scatter_(0, good_triangle_indices, False)
        
        assert good_triangle_indices.size(0) == incircle_test.logical_not().sum()
        assert bad_triangle_indices_mask.sum() == batch_newtriangles
        
        partial_delaunay_triangles[good_triangle_indices] = self.partial_delaunay_triangles[~incircle_test]
        batch_index[good_triangle_indices] = self.batch_index[~incircle_test]
        
        partial_delaunay_triangles[bad_triangle_indices_mask] = newtriangles
        batch_index[bad_triangle_indices_mask] = nomatch_batch_index
        
        self.partial_delaunay_triangles = partial_delaunay_triangles
        self.batch_index = batch_index
        
        self.ntriangles = ntriangles
        self.batch_triangles = self.partial_delaunay_triangles.size(0)
        
        self.points_mask.scatter_(
            1, point_index.unsqueeze(1).expand(-1, self.npoints), True
        )
        self.points_sequence = torch.cat(
            [self.points_sequence, point_index.unsqueeze(1)], dim=1
        )
        
        self.cost += nremoved_triangles
        return
    
    def sample_point(self, logits): #logits must be [batchsize * nsamples, npoints]
        probs = torch.distributions.categorical.Categorical(logits = logits)
        next_point = probs.sample() #size is [batchsize * nsamples]
        self.update(next_point)
        self.logprob = self.logprob + probs.log_prob(next_point)
        return next_point #[batchsize * nsamples]
    
    def sampleandgreedy_point(self, logits): #logits must be [batchsize * nsamples, npoints], last sample will be the greedy choice (but we still need to keep track of its logits)
        logits_sample = logits.view(-1, self.nsamples, self.npoints)[:, :-1, :]
        probs = torch.distributions.categorical.Categorical(logits = logits_sample)
        
        sample_point = probs.sample() #[batchsize, (nsamples - 1)]
        greedy_point = logits.view(-1, self.nsamples, self.npoints)[:, -1, :].max(-1, keepdim = True)[1] #[batchsize, 1]
        next_point = torch.cat([sample_point, greedy_point], dim = 1).view(-1)
        self.update(next_point)
        self.logprob = self.logprob + torch.cat([probs.log_prob(sample_point), torch.zeros([sample_point.size(0), 1], device = device)], dim = 1).view(-1)
        return next_point
    

env = environment()


def train(epochs = 30000, npoints = 14, batchsize = 100, nsamples = 8):
    NN.train()
    for i in range(epochs):
        env.reset(npoints, batchsize, nsamples)
        """include the boundary points, kinda makes sense that they should contribute (atm only in the encoder, difficult to see how in the decoder)"""
        memory = NN.encode(env.points) #[npoints, batchsize * nsamples, emsize]
        #### #### #### remember to include tgt.detach() when reinstate with torch.no_grad()
        tgt = NN.start_token.unsqueeze(0).unsqueeze(1).expand(1, batchsize * nsamples, -1).detach() #[1, batchsize * nsamples, emsize]
        #with torch.no_grad(): #to speed up computation, selecting routes is done without gradient
        with torch.no_grad():
            for j in range(4, npoints):
                #### #### #### remember to include memory.detach() when reinstate with torch.no_grad()
                _, logits = NN.decode_next(memory.detach(), tgt, env.points_mask)
                next_point = env.sampleandgreedy_point(logits)
                """
                for inputing the previous embedding into decoder
                """
                tgt = torch.cat([tgt, memory.gather(0, next_point.unsqueeze(0).unsqueeze(2).expand(1, -1, memory.size(2)))]) #[nsofar, batchsize * nsamples, emsize]
                """
                for inputing the previous decoder output into the decoder (allows for an evolving strategy, but doesn't allow for fast training
                """
                ############

        
        NN.eval()
        _, logprob = NN.calculate_logprob(memory, env.points_sequence) #[batchsize * nsamples]
        NN.train()
        """
        clip logprob so doesn't reinforce things it already knows
        TBH WANT SOMETHING DIFFERENT ... want to massively increase training if find something unexpected and otherwise not
        """
        greedy_prob = logprob.view(batchsize, nsamples)[:, -1].detach() #[batchsize]
        greedy_baseline = env.cost.view(batchsize, nsamples)[:, -1] #[batchsize], greedy sample
        fixed_baseline = 0.5 * torch.ones([1], device = device)
        min_baseline = env.cost.view(batchsize, nsamples)[:, :-1].min(-1)[0] #[batchsize], minimum cost
        baseline = greedy_baseline
        positive_reinforcement = - F.relu( - (env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))) #don't scale positive reinforcement
        negative_reinforcement = F.relu(env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))
        positive_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) <= -0.05
        negative_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) > 1
        """
        binary positive reinforcement
        """
        #loss = - ((logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement_binary).mean() #+ (logprob.view(batchsize, nsamples)[:, :-1] > -1) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement_binary
        """
        clipped binary reinforcement
        """
        #loss = ( 
        #        - logprob.view(batchsize, nsamples)[:, :-1] 
        #        #* (logprob.view(batchsize, nsamples)[:, :-1] < 0) 
        #        * positive_reinforcement_binary 
        #        + logprob.view(batchsize, nsamples)[:, :-1] 
        #        #* (logprob.view(batchsize, nsamples)[:, :-1] > greedy_prob.unsqueeze(1) - 13) 
        #        * negative_reinforcement_binary 
        #).mean()
        """
        clipped binary postive, clipped weighted negative
        """
        #loss = ( - logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * positive_reinforcement_binary + logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] > -2) * negative_reinforcement ).mean()
        """
        clipped reinforcement without rescaling
        """
        #loss = ((logprob.view(batchsize, nsamples)[:, :-1] < -0.7) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement + (logprob.view(batchsize, nsamples)[:, :-1] > -5) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement).mean()
        """
        clipped reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + (logprob.view(batchsize, nsamples)[:, :-1] > -3) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt()).mean()
        """
        balanced reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt())).mean()
        """
        regular loss
        """
        loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement + negative_reinforcement)).mean()
        optimizer.zero_grad()
        loss.backward()
        #print(NN.encoder.weight.grad)
        optimizer.step()
        #print(greedy_baseline.mean().item())
        print(greedy_baseline.mean().item(), logprob.view(batchsize, nsamples)[:, -1].mean().item(), logprob.view(batchsize, nsamples)[:, :-1].mean().item(), logprob[batchsize - 1].item(), logprob[0].item(), env.logprob[0].item())
        
def evaluate(epochs = 30000, npoints = 14, batchsize = 100, nsamples = 8):
    NN.eval()
    for i in range(epochs):
        env.reset(npoints, batchsize, nsamples)
        """include the boundary points, kinda makes sense that they should contribute (atm only in the encoder, difficult to see how in the decoder)"""
        memory = NN.encode(env.points) #[npoints, batchsize * nsamples, emsize]
        #### #### #### remember to include tgt.detach() when reinstate with torch.no_grad()
        tgt = NN.start_token.unsqueeze(0).unsqueeze(1).expand(1, batchsize * nsamples, -1).detach() #[1, batchsize * nsamples, emsize]
        #with torch.no_grad(): #to speed up computation, selecting routes is done without gradient
        with torch.no_grad():
            for j in range(4, npoints):
                #### #### #### remember to include memory.detach() when reinstate with torch.no_grad()
                _, logits = NN.decode_next(memory.detach(), tgt, env.points_mask)
                next_point = env.sampleandgreedy_point(logits)
                """
                for inputing the previous embedding into decoder
                """
                tgt = torch.cat([tgt, memory.gather(0, next_point.unsqueeze(0).unsqueeze(2).expand(1, -1, memory.size(2)))]) #[nsofar, batchsize * nsamples, emsize]
                """
                for inputing the previous decoder output into the decoder (allows for an evolving strategy, but doesn't allow for fast training
                """
                ############
        
        
        
        _, logprob = NN.calculate_logprob(memory, env.points_sequence) #[batchsize * nsamples]
        NN.train()
        """
        clip logprob so doesn't reinforce things it already knows
        TBH WANT SOMETHING DIFFERENT ... want to massively increase training if find something unexpected and otherwise not
        """
        
        greedy_baseline = env.cost.view(batchsize, nsamples)[:, -1] #[batchsize], greedy sample
        
        print(greedy_baseline.mean().item(), logprob.view(batchsize, nsamples)[:, -1].mean().item(), logprob.view(batchsize, nsamples)[:, :-1].mean().item(), logprob[batchsize - 1].item(), logprob[0].item(), env.logprob[0].item())
  

In [2]:
evaluate(1, 53, 100)

610.5599975585938 -141.84889221191406 -144.45701599121094 -144.08462524414062 -144.19662475585938 -144.19662475585938


In [3]:
NN.load_state_dict(torch.load('3D_100points_small_1230'))

<All keys matched successfully>

In [2]:
def evaluate(epochs = 30000, npoints = 14, batchsize = 100, nsamples = 8):
    NN.eval()
    with torch.no_grad():
        for i in range(epochs):
            env.reset(npoints, batchsize, nsamples)
            """include the boundary points, kinda makes sense that they should contribute (atm only in the encoder, difficult to see how in the decoder)"""
            memory = NN.encode(env.points) #[npoints, batchsize * nsamples, emsize]
            #### #### #### remember to include tgt.detach() when reinstate with torch.no_grad()
            tgt = NN.start_token.unsqueeze(0).unsqueeze(1).expand(1, batchsize * nsamples, -1).detach() #[1, batchsize * nsamples, emsize]
            #with torch.no_grad(): #to speed up computation, selecting routes is done without gradient
            for j in range(4, npoints):
                #### #### #### remember to include memory.detach() when reinstate with torch.no_grad()
                _, logits = NN.decode_next(memory.detach(), tgt, env.points_mask)
                next_point = env.sampleandgreedy_point(logits)
                """
                for inputing the previous embedding into decoder
                """
                tgt = torch.cat([tgt, memory.gather(0, next_point.unsqueeze(0).unsqueeze(2).expand(1, -1, memory.size(2)))]) #[nsofar, batchsize * nsamples, emsize]
                """
                for inputing the previous decoder output into the decoder (allows for an evolving strategy, but doesn't allow for fast training
                """
                ############


            NN.eval()
            _, logprob = NN.calculate_logprob(memory, env.points_sequence) #[batchsize * nsamples]
            NN.train()
            """
            clip logprob so doesn't reinforce things it already knows
            TBH WANT SOMETHING DIFFERENT ... want to massively increase training if find something unexpected and otherwise not
            """
            
            greedy_baseline = env.cost.view(batchsize, nsamples)[:, -1] #[batchsize], greedy sample
            
            print(greedy_baseline.mean().item(), logprob.view(batchsize, nsamples)[:, -1].mean().item(), logprob.view(batchsize, nsamples)[:, :-1].mean().item(), logprob[batchsize - 1].item(), logprob[0].item(), env.logprob[0].item())
        
#evaluate(epochs = 10, npoints = 24, batchsize = 100, nsamples = 2)

In [3]:
evaluate(10, 1004, 1, 1)

26674.0 -5835.2109375 nan -5835.2109375 -5835.2109375 0.0
26598.0 -5832.29833984375 nan -5832.29833984375 -5832.29833984375 0.0
26653.0 -5835.2451171875 nan -5835.2451171875 -5835.2451171875 0.0
26557.0 -5838.02685546875 nan -5838.02685546875 -5838.02685546875 0.0
26723.0 -5836.95703125 nan -5836.95703125 -5836.95703125 0.0
26678.0 -5835.369140625 nan -5835.369140625 -5835.369140625 0.0
26694.0 -5833.19921875 nan -5833.19921875 -5833.19921875 0.0
26587.0 -5833.3115234375 nan -5833.3115234375 -5833.3115234375 0.0
26701.0 -5832.5869140625 nan -5832.5869140625 -5832.5869140625 0.0
26702.0 -5837.625 nan -5837.625 -5837.625 0.0


In [1]:
"""
REDO THIS WITHOUT KEEPING TRACK OF EDGES

Idea: among removed triangles, pair up faces that both apear, left with faces that don't - the boundary, from which we construct new triangles

have two lists, faces left to check, and faces to check against (these will be all 3 anticlockwise versions of each face)
keep track of the batch you came from, and the index against which you are currently checking
increase index by one each time until either: find a match, or: no longer checking against same batch
at which point we remove FROM THE FIRST LIST
repeat until all removed
when find a match, mark it in second list
removed all marked faces
somehow find number remaining in each batch, and make sure to copy that many 'new points' into a long list
construct new triangles from the above information
"""



import math
import random
import numpy as np
import torch


device = "cuda:0"
floattype = torch.float


class environment:    
    def reset(self, npoints, batchsize, nsamples=1, corner_points = None, initial_triangulation = None):
        """
        corner_points, etc., shoudn't include a batch dimension
        """
        if corner_points == None:
            ncornerpoints = 4
        else:
            ncornerpoints = corner_points.size(0)
        if npoints <= ncornerpoints:
            print("Error: not enough points for valid problem instance")
            return
        self.batchsize = (
            batchsize * nsamples
        )  # so that I don't have to rewrite all this code, we store these two dimensions together
        self.nsamples = nsamples
        self.npoints = npoints
        self.points = (
            torch.rand([batchsize, npoints - ncornerpoints, 3], dtype = floattype, device=device)
            .unsqueeze(1)
            .expand(-1, nsamples, -1, -1)
            .reshape(self.batchsize, npoints - ncornerpoints, 3)
        )
        if corner_points == None:
            self.corner_points = torch.tensor(
                [[0, 0, 0], [3, 0, 0], [0, 3, 0], [0, 0, 3]], dtype = floattype, device=device
            )
        else:
            self.corner_points = corner_points
        self.points = torch.cat(
            [
                self.corner_points.unsqueeze(0).expand(self.batchsize, -1, -1),
                self.points,
            ],
            dim=-2,
        )  # [batchsize * nsamples, npoints, 3]
        self.points_mask = torch.cat(
            [
                torch.ones([self.batchsize, ncornerpoints], dtype=torch.bool, device=device),
                torch.zeros(
                    [self.batchsize, npoints - ncornerpoints], dtype=torch.bool, device=device
                ),
            ],
            dim=1,
        )
        self.points_sequence = torch.empty(
            [self.batchsize, 0], dtype=torch.long, device=device
        )

        """
        points are now triples
        triangles are now quadruples
        edges are now still just indices, but there are four of them per 'triangle', and they correspond to triples of points, not pairs
        we use  0,2,1  0,3,2  0,1,3  1,2,3  as the order of the four 'edges'/faces
        opposite face is always ordered such that the last two indices are swapped
        faces are always read ANTICLOCKWISE
        
        first three points of tetrahedron MUST be read clockwise (from the outside) to get correct sign on incircle test
        
        new point will be inserted in zeroth position, so if corresponding face of REMOVED tetrahedron is [x,y,z] (being read anticlockwise from outside in) new tetrahedron is [p, x, y, z]
        """
        
        """
        number of tetrahedra is not the same for each batch (in 3D), so store as a big list, and remember batch index that it comes from
        """
        if corner_points == None:
            initial_triangulation = torch.tensor([[0, 1, 2, 3]], dtype=torch.long, device=device)
        
        self.partial_delaunay_triangles = initial_triangulation.unsqueeze(0).expand(self.batchsize, -1, -1).reshape(-1, 4)
        self.batch_index = torch.arange(self.batchsize, dtype = torch.long, device = device).unsqueeze(1).expand(-1, initial_triangulation.size(0)).reshape(-1)
        
        self.batch_triangles = self.partial_delaunay_triangles.size(0) #[0]
        self.ntriangles = torch.full([self.batchsize], initial_triangulation.size(0), dtype = torch.long, device = device) #[self.batchsize]
        
        self.cost = torch.zeros([self.batchsize], dtype = floattype, device=device)

        self.logprob = torch.zeros([self.batchsize], dtype = floattype, device=device, requires_grad=True)

    def update(self, point_index):  # point_index is [batchsize]
        
        assert point_index.size(0) == self.batchsize
        assert str(point_index.device) == device
        assert self.points_mask.gather(1, point_index.unsqueeze(1)).sum() == 0
        
        triangles_coordinates = self.points[self.batch_index.unsqueeze(1), self.partial_delaunay_triangles] # [batch_triangles, 4, 3]
        
        newpoint = self.points[self.batch_index, point_index[self.batch_index]] # [batch_triangles, 3]

        incircle_matrix = torch.cat(
            [
                newpoint.unsqueeze(1),
                triangles_coordinates,
            ],
            dim=-2,
        )  # [batch_triangles, 5, 3]
        incircle_matrix = torch.cat(
            [
                (incircle_matrix * incircle_matrix).sum(-1, keepdim=True),
                incircle_matrix,
                torch.ones([self.batch_triangles, 5, 1], dtype = floattype, device=device),
            ],
            dim=-1,
        )  # [batch_triangles, 5, 5]
        assert incircle_matrix.dtype == floattype
        assert str(incircle_matrix.device) == device
        
        incircle_test = (
            incircle_matrix.det() > 0
        )  # [batch_triangles], is True if inside incircle
        
        conflicts = incircle_test.sum()
        
        conflicting_triangles = self.partial_delaunay_triangles[incircle_test] # [conflicts, 4]
        
        conflicting_edges_index0 = torch.empty_like(conflicting_triangles)
        indices = torch.LongTensor([0, 0, 0, 1])
        conflicting_edges_index0 = conflicting_triangles[:, indices] # [conflicts, 4]
        
        conflicting_edges_index1 = torch.empty_like(conflicting_triangles)
        indices = torch.LongTensor([2, 3, 1, 2])
        conflicting_edges_index1 = conflicting_triangles[:, indices] # [conflicts, 4]
        
        conflicting_edges_index2 = torch.empty_like(conflicting_triangles)
        indices = torch.LongTensor([1, 2, 3, 3])
        conflicting_edges_index2 = conflicting_triangles[:, indices] # [conflicts, 4]
        
        conflicting_edges = torch.cat([conflicting_edges_index0.view(-1).unsqueeze(-1), conflicting_edges_index1.view(-1).unsqueeze(-1), conflicting_edges_index2.view(-1).unsqueeze(-1)], dim = -1).reshape(-1, 3) # [conflicts * 4, 3]
        
        edge_batch_index = self.batch_index[incircle_test].unsqueeze(1).expand(-1, 4).reshape(-1) # [conflicts * 4]
        
        indices = torch.LongTensor([0, 2, 1])
        comparison_edges = conflicting_edges[:, indices] # [conflicts * 4, 3]        
        
        unravel_nomatch_mask = torch.ones([conflicts * 4], dtype = torch.bool, device = device) # [conflicts * 4]
        i = 1
        while True:
            
            todo_mask = unravel_nomatch_mask[:-i].logical_and(edge_batch_index[:-i] == edge_batch_index[i:])
            if i % 4 == 0:
                if todo_mask.sum() == 0:
                    break
            
            match_mask = todo_mask.clone()
            match_mask[todo_mask] = (conflicting_edges[:-i][todo_mask] != comparison_edges[i:][todo_mask]).sum(-1).logical_not()
            
            unravel_nomatch_mask[:-i][match_mask] = False
            unravel_nomatch_mask[i:][match_mask] = False
            
            i += 1
        
        batch_newtriangles = unravel_nomatch_mask.sum()
        
        nomatch_edges = conflicting_edges[unravel_nomatch_mask] # [batch_newtriangles, 3], already in correct order to insert into 1,2,3 (since already anticlockwise from outside in)
        assert list(nomatch_edges.size()) == [batch_newtriangles, 3]
        nomatch_batch_index = edge_batch_index[unravel_nomatch_mask] # [batch_newtriangles]
        
        nomatch_newpoint = point_index[nomatch_batch_index] # [batch_newtriangles]
        
        newtriangles = torch.cat([nomatch_newpoint.unsqueeze(1), nomatch_edges], dim = -1) # [batch_newtriangles, 4]
        
        
        nremoved_triangles = torch.zeros([self.batchsize], dtype = torch.long, device = device)
        nnew_triangles = torch.zeros([self.batchsize], dtype = torch.long, device = device)
        
        indices = self.batch_index[incircle_test]
        nremoved_triangles.put_(indices, torch.ones_like(indices, dtype = torch.long), accumulate = True) # [batchsize]
        
        indices = edge_batch_index[unravel_nomatch_mask]
        nnew_triangles.put_(indices, torch.ones_like(indices, dtype = torch.long), accumulate = True) # [batchsize]
        
        assert (nnew_triangles <= 2 * nremoved_triangles + 2).logical_not().sum().logical_not()
        
        """
        NOTE:
        I THINK it's possible for nnew_triangles to be less than nremoved_triangles (or my code is just buggy...)
        """
        
        assert nnew_triangles.sum() == batch_newtriangles
        assert nremoved_triangles.sum() == incircle_test.sum()
        
        nadditional_triangles = nnew_triangles - nremoved_triangles # [batchsize]
        ntriangles = self.ntriangles + nadditional_triangles # [batchsize]
        
        partial_delaunay_triangles = torch.empty([ntriangles.sum(), 4], dtype = torch.long, device = device)
        batch_index = torch.empty([ntriangles.sum()], dtype = torch.long, device = device)
        
        cumulative_triangles = torch.cat([torch.zeros([1], dtype = torch.long, device = device), nnew_triangles.cumsum(0)[:-1]]) # [batchsize], cumulative sum starts at zero
        
        """
        since may actually have LESS triangles than previous round, we insert all that survive into the first slots (in that batch)
        """
        good_triangle_indices = torch.arange(incircle_test.logical_not().sum(), dtype = torch.long, device = device)
        good_triangle_indices += cumulative_triangles[self.batch_index[incircle_test.logical_not()]]
        bad_triangle_indices_mask = torch.ones([ntriangles.sum(0)], dtype = torch.bool, device = device)
        bad_triangle_indices_mask.scatter_(0, good_triangle_indices, False)
        
        assert good_triangle_indices.size(0) == incircle_test.logical_not().sum()
        assert bad_triangle_indices_mask.sum() == batch_newtriangles
        
        partial_delaunay_triangles[good_triangle_indices] = self.partial_delaunay_triangles[~incircle_test]
        batch_index[good_triangle_indices] = self.batch_index[~incircle_test]
        
        partial_delaunay_triangles[bad_triangle_indices_mask] = newtriangles
        batch_index[bad_triangle_indices_mask] = nomatch_batch_index
        
        self.partial_delaunay_triangles = partial_delaunay_triangles
        self.batch_index = batch_index
        
        self.ntriangles = ntriangles
        self.batch_triangles = self.partial_delaunay_triangles.size(0)
        
        self.points_mask.scatter_(
            1, point_index.unsqueeze(1).expand(-1, self.npoints), True
        )
        self.points_sequence = torch.cat(
            [self.points_sequence, point_index.unsqueeze(1)], dim=1
        )
        
        self.cost += nremoved_triangles
        return
    
    def allindices(self): #generate all orders of point insertion
        npoints = self.npoints - 4
        allroutes = torch.empty([1, 0], dtype = torch.long, device = device)
        for i in range(npoints):
            nroutes = allroutes.size(0)
            remaining_mask = torch.ones([nroutes], dtype = torch.bool, device = device).unsqueeze(1).expand(-1, npoints).clone().scatter_(-1, allroutes, False)
            remaining_indices = remaining_mask.nonzero(as_tuple = True)[1]
            allroutes = allroutes.unsqueeze(1).expand(-1, remaining_mask[0, :].sum(), -1)
            allroutes = torch.cat([allroutes, remaining_indices.view(nroutes, -1).unsqueeze(2)], dim = -1).view(-1, allroutes.size(-1) + 1)
        return allroutes #[npoints!, npoints]


# turn integer x,y coords (in nxn grid) into position d (0 to n^2-1) along the Hilbert curve.
def xy2d(n, x, y):
    [x, y] = [math.floor(x), math.floor(y)]
    [rx, ry, s, d] = [0, 0, 0, 0]
    s = n / 2
    s = math.floor(s)
    while s > 0:
        rx = (x & s) > 0  # bitwise and, and then boolean is it greater than 0?
        ry = (y & s) > 0
        d += s * s * ((3 * rx) ^ ry)
        [x, y] = rot(n, x, y, rx, ry)
        s = s / 2
        s = math.floor(s)
    return d


def rot(n, x, y, rx, ry):
    if ry == 0:
        if rx == 1:
            x = n - 1 - x
            y = n - 1 - y
        # Swap x and y
        t = x
        x = y
        y = t
    return x, y


def order(
    n, points
):  # turns tensor of points into integer distances along hilbert curve of itteration n
    grid = n * points.to("cpu")
    x = torch.empty([grid.size(0)])
    for i in range(points.size(0)):
        x[i] = xy2d(n, grid[i, 0], grid[i, 1])
    return x

"""
CURRENTLY ONLY 2D VERSION
"""
def hilbert_insertion(npoints=103, batchsize=200):
    env.reset(npoints, batchsize)
    points = env.points[:, 3:]  # [batchsize, npoints - 3]
    insertion_order = torch.full([batchsize, npoints], float("inf"), device=device)
    for i in range(batchsize):
        insertion_order[i, 3:] = order(
            2 ** 6, points[i]
        )  # number of possible positions is n ** 2
    for i in range(npoints - 3):
        next_index = insertion_order.min(-1)[1]
        env.update(next_index)
        insertion_order.scatter_(1, next_index.unsqueeze(1), float("inf"))
    print(env.cost.mean().item(), env.cost.var().sqrt().item())
    return


def random_insertion(npoints=104, batchsize=200):
    env.reset(npoints, batchsize)
    for i in range(npoints - 4):
        env.update(torch.full([batchsize], i + 4, dtype=torch.long, device=device))
    print(env.cost.mean().item(), env.cost.var().sqrt().item())
    return

    """
    UNDER CONSTRUCTION
    """
def kdtree_insertion(npoints=103, batchsize=200):
    env.reset(npoints, batchsize)
    points = env.points[:, 3:]  # [batchsize, npoints - 3]
    
env = environment()

In [2]:
env = environment()
npoints = 9
batchsize = 1
        
env.reset(npoints + 4, batchsize, math.factorial(npoints))
allroutes = env.allindices() + 4
allroutes = allroutes.unsqueeze(0).expand(batchsize, -1, -1).reshape(-1, npoints)
for j in range(10):
    for i in range(npoints):
        env.update(allroutes[:, i])
    print(env.cost.view(batchsize, -1).min(-1)[0].mean().item(), env.cost.mean().item())
    env.reset(npoints + 4, batchsize, math.factorial(npoints))

25.0 43.19682312011719
27.0 42.42301559448242
26.0 43.520633697509766
26.0 41.43492126464844
30.0 40.770633697509766
32.0 42.38730239868164
26.0 39.82460403442383
31.0 41.69682312011719
28.0 40.746826171875
28.0 42.14603042602539


In [2]:
env = environment()
npoints = 8
batchsize = 10
        
env.reset(npoints + 4, batchsize, math.factorial(npoints))
allroutes = env.allindices() + 4
allroutes = allroutes.unsqueeze(0).expand(batchsize, -1, -1).reshape(-1, npoints)
for j in range(10):
    for i in range(npoints):
        env.update(allroutes[:, i])
    print(env.cost.view(batchsize, -1).min(-1)[0].mean().item(), env.cost.mean().item())
    env.reset(npoints + 4, batchsize, math.factorial(npoints))

23.5 34.148094177246094
23.200000762939453 34.55595016479492
23.0 34.28666687011719
22.700000762939453 33.59000015258789
21.80000114440918 34.04595184326172
23.0 33.55976104736328
23.600000381469727 34.18499755859375
22.899999618530273 33.813331604003906
22.200000762939453 34.28190231323242
22.100000381469727 34.10976028442383


In [ ]:
env = environment()
npoints = 10
batchsize = 1
nsamples = 100000
        
env.reset(npoints + 4, nsamples)
allroutes = env.allindices() + 4
allroutes = allroutes.unsqueeze(0).expand(batchsize, -1, -1).reshape(-1, npoints)[:nsamples, :]
print(allroutes.size())
for j in range(1000):
    for i in range(npoints):
        env.update(allroutes[:, i])
    print(env.cost.view(batchsize, -1).min(-1)[0].mean().item(), env.cost.mean().item())
    env.reset(npoints + 4, batchsize, nsamples)

torch.Size([100000, 10])
29.0 49.65717697143555
36.0 48.87055969238281
38.0 51.05009841918945
36.0 51.02914047241211
31.0 45.161277770996094
41.0 52.800697326660156
38.0 49.204898834228516
39.0 50.78003692626953
38.0 49.051597595214844
33.0 47.717559814453125
32.0 46.3649787902832
35.0 47.345619201660156
35.0 45.504600524902344
35.0 49.83915710449219
33.0 44.56079864501953
37.0 51.07265853881836
37.0 52.16703796386719
38.0 49.48624038696289
36.0 51.30870056152344
35.0 46.69947814941406
39.0 53.22419738769531
33.0 48.71516036987305
36.0 52.49327850341797
38.0 51.98114013671875
44.0 52.908878326416016
34.0 49.60095977783203
35.0 50.536598205566406
35.0 51.26199722290039
38.0 49.712860107421875
36.0 53.11921691894531
36.0 49.411720275878906
33.0 49.54199981689453
35.0 47.198917388916016
39.0 49.790977478027344
36.0 54.170318603515625
37.0 49.390159606933594
38.0 49.5429573059082
37.0 48.32611846923828
39.0 51.07919692993164
35.0 49.789737701416016
35.0 51.28583908081055
35.0 47.7252006530

In [2]:
for i in range(10):
    random_insertion(1004, 10)

17675.400390625 274.7860412597656
17795.80078125 259.2129821777344
17695.30078125 179.09890747070312
17705.099609375 249.88575744628906
17682.30078125 188.93975830078125
17710.599609375 286.4358215332031
17809.400390625 172.30747985839844
17747.30078125 254.40084838867188
17731.80078125 197.7528076171875
17709.400390625 200.54603576660156


In [1]:
import math
import random
import numpy as np
import torch


"""
Install pytorch first, see https://pytorch.org/get-started/locally/ for instructions
"""


"""
If a GPU is available set device to "cuda" below, and this will speed up the code significantly (be careful not to overflow the GPU memory)
To find out if a GPU is available you can use the command
    torch.cuda.is_available()
"""
device = "cuda:0"


"""
To just compute the cost of the Hilbert Curve alogrithm use the function
    hilbert_insertion(npoints, batchsize)
npoints is the total number of points (including the 3 original corner points!!!!)
batchsize is the number of problem instances we triangulate (say 200, then we would have 200 sets of npoints)

This function will just print the average number of 'additional' triangles deleted, and the standard deviation of the number of 'additional' triangles deleted
"""


"""
To just compute the cost of the random insertion alogrithm use the function
    random_insertion(npoints, batchsize)
npoints is the total number of points (including the 3 original corner points!!!!)
batchsize is the number of problem instances we triangulate (say 200, then we would have 200 sets of npoints)

this function will just print the average number of 'additional' triangles deleted, and the standard deviation of the number of 'additional' triangles deleted
"""


"""
This class (below) is the actual delaunay algorithm

Initiate by
    env = environment()
To generate new random points use
    env.reset(npoints, batchsize)
(don't worry about what nsamples does in the actual code below, it's used for training NNs using reinforcement learning)

npoints is the total number of points to be generated, the first 3 are the 3 corners of the big triangle, all the rest are randomly chosen in the unit square
batchsize is how many problems you want to simultaneously create (lets say 200), then you would have 200 sets of npoints, currently all 200 are untriangulated

The current point locations can be found by
    env.points
The current (partial) delaunay triangulations can be found by
    env.triangles
These are stored as triples of indices, so the triangle [0, 2426, 564] refers to the 0th point, 2426th point, and 564th point stored in env.points
Note: the big outside triangle has two copies, a clockwise ([2, 1, 0]) and anticlockwise ([0, 1, 2]) one, the clockwise one will always stay and never get removed - it's used for some trickery for when the delaunay cavity includes one of the external edges, don't worry about it

To add a point to the current (partial) delaunay triangulation use
    env.update(points)
where points is a torch.tensor (see the pytorch website for how to generate and use tensors) of size [batchsize], containing the index of the points you want to add FOR EACH PROBLEM INSTANCE (remember there might be batchsize = 200 problem instances, so you would have to specify 200 points to add, one for each problem instance)
This will update the current (partial) delaunay triangulation, env.triangles

To see the current cost (the 'additional' triangles that had to be removed) use
    env.cost
This will output a torch.tensor of size [batchsize], i.e. a 'list' of costs, corresponding to each of the (say 200) problem instances
"""


class environment:
    def reset(self, npoints, batchsize, nsamples=1):
        if npoints <= 3:
            print("Error: not enough points for valid problem instance")
            return
        self.batchsize = (
            batchsize * nsamples
        )  # so that I don't have to rewrite all this code, we store these two dimensions together
        self.nsamples = nsamples
        self.npoints = npoints
        self.points = (
            torch.rand([batchsize, npoints - 3, 2], device=device)
            .unsqueeze(1)
            .expand(-1, nsamples, -1, -1)
            .reshape(self.batchsize, npoints - 3, 2)
        )
        self.corner_points = torch.tensor(
            [[0, 0], [2, 0], [0, 2]], dtype=torch.float, device=device
        )
        self.points = torch.cat(
            [
                self.corner_points.unsqueeze(0).expand(self.batchsize, -1, -1),
                self.points,
            ],
            dim=-2,
        )  # [batchsize * nsamples, npoints, 2]
        self.points_mask = torch.cat(
            [
                torch.ones([self.batchsize, 3], dtype=torch.bool, device=device),
                torch.zeros(
                    [self.batchsize, npoints - 3], dtype=torch.bool, device=device
                ),
            ],
            dim=1,
        )
        self.points_sequence = torch.empty(
            [self.batchsize, 0], dtype=torch.long, device=device
        )

        """use a trick, for the purpose of an 'external' triangle that is always left untouched, which means we don't have to deal with boundary edges as being different. external triangle is [0, 1, 2] traversed clockwise..."""
        self.partial_delaunay_triangles = (
            torch.tensor([[0, 2, 1], [0, 1, 2]], dtype=torch.int64, device=device)
            .unsqueeze(0)
            .expand(self.batchsize, -1, -1)
            .contiguous()
        )  # [batchsize, ntriangles, 3] contains index of points, always anticlockwise
        self.partial_delaunay_edges = (
            torch.tensor([5, 4, 3, 2, 1, 0], dtype=torch.int64, device=device)
            .unsqueeze(0)
            .expand(self.batchsize, -1)
            .contiguous()
        )  # [batchsize, ntriangles * 3] contains location of corresponding edge (edges go in order 01, 12, 20). Edges will always flip since triangles are stored anticlockwise.

        self.ntriangles = 2  # can store as scalar, since will always be the same
        self.cost = torch.zeros([self.batchsize], device=device)

        self.logprob = torch.zeros([self.batchsize], device=device, requires_grad=True)

    def update(self, point_index):  # point_index is [batchsize]
        if point_index.size(0) != self.batchsize:
            print(
                "Error: point_index.size() doesn't match expected size, should be [batchsize]"
            )
            return
        if self.points_mask.gather(1, point_index.unsqueeze(1)).sum():
            print("Error: some points already added")
            return
        triangles_coordinates = self.points.gather(
            1,
            self.partial_delaunay_triangles.view(self.batchsize, self.ntriangles * 3)
            .unsqueeze(2)
            .expand(-1, -1, 2),
        ).view(
            self.batchsize, self.ntriangles, 3, 2
        )  # [batchsize, ntriangles, 3, 2]
        newpoint = self.points.gather(
            1, point_index.unsqueeze(1).unsqueeze(2).expand(self.batchsize, 1, 2)
        ).squeeze(
            1
        )  # [batchsize, 2]

        incircle_matrix = torch.cat(
            [
                triangles_coordinates,
                newpoint.unsqueeze(1).unsqueeze(2).expand(-1, self.ntriangles, 1, -1),
            ],
            dim=-2,
        )  # [batchsize, ntriangles, 4, 2]
        incircle_matrix = torch.cat(
            [
                incircle_matrix,
                (incircle_matrix * incircle_matrix).sum(-1, keepdim=True),
                torch.ones([self.batchsize, self.ntriangles, 4, 1], device=device),
            ],
            dim=-1,
        )  # [batchsize, ntriangles, 4, 4]
        incircle_test = (
            incircle_matrix.det() > 0
        )  # [batchsize, ntriangles], is True if inside incircle
        removed_edge_mask = (
            incircle_test.unsqueeze(2).expand(-1, -1, 3).reshape(-1)
        )  # [batchsize * ntriangles * 3]

        edges = (
            self.partial_delaunay_edges
            + self.ntriangles
            * 3
            * torch.arange(self.batchsize, device=device).unsqueeze(1)
        ).view(
            -1
        )  # [batchsize * ntriangles * 3]
        neighbouring_edge = edges.masked_select(removed_edge_mask)
        neighbouring_edge_mask = torch.zeros(
            [self.batchsize * self.ntriangles * 3], device=device, dtype=torch.bool
        )
        neighbouring_edge_mask[neighbouring_edge] = True
        neighbouring_edge_mask = (
            neighbouring_edge_mask * removed_edge_mask.logical_not()
        )  # [batchsize * ntriangles * 3]

        n_new_triangles = neighbouring_edge_mask.view(self.batchsize, -1).sum(
            -1
        )  # [batchsize]

        new_point = (
            point_index.unsqueeze(1)
            .expand(-1, self.ntriangles * 3)
            .masked_select(neighbouring_edge_mask.view(self.batchsize, -1))
        )

        second_point_mask = neighbouring_edge_mask.view(
            self.batchsize, -1, 3
        )  # [batchsize, ntriangles 3]
        (
            first_point_indices0,
            first_point_indices1,
            first_point_indices2,
        ) = second_point_mask.nonzero(as_tuple=True)
        first_point_indices2 = (first_point_indices2 != 2) * (first_point_indices2 + 1)

        first_point = self.partial_delaunay_triangles[
            first_point_indices0, first_point_indices1, first_point_indices2
        ]  # [?]
        second_point = self.partial_delaunay_triangles.masked_select(
            second_point_mask
        )  # [?]

        new_triangles_mask = torch.cat(
            [
                incircle_test,
                torch.ones([self.batchsize, 2], dtype=torch.bool, device=device),
            ],
            dim=1,
        )  # [batchsize, ntriangles + 2]

        new_neighbouring_edges = (
            3 * new_triangles_mask.nonzero(as_tuple=True)[1]
        )  # [?], 3* since is the 01 edge of new triangles (see later)
        self.partial_delaunay_edges.masked_scatter_(
            neighbouring_edge_mask.view(self.batchsize, -1), new_neighbouring_edges
        )  # still [batchsize, ntriangles * 3] for now

        self.partial_delaunay_triangles = torch.cat(
            [
                self.partial_delaunay_triangles,
                torch.empty([self.batchsize, 2, 3], dtype=torch.long, device=device),
            ],
            dim=1,
        )
        self.partial_delaunay_edges = torch.cat(
            [
                self.partial_delaunay_edges,
                torch.empty([self.batchsize, 6], dtype=torch.long, device=device),
            ],
            dim=1,
        )
        new_triangles = torch.stack(
            [first_point, second_point, new_point], dim=1
        )  # [?, 3], edge here is flipped compared to edge in neighbouring triangle (so first_point is the second point in neighbouring edge)
        self.partial_delaunay_triangles.masked_scatter_(
            new_triangles_mask.unsqueeze(2).expand(-1, -1, 3), new_triangles
        )  # [batchsize, ntriangles + 2, 3]

        new_edge01 = neighbouring_edge_mask.view(self.batchsize, -1).nonzero(
            as_tuple=True
        )[
            1
        ]  # [?]

        """we are currently storing which triangles have to be inserted, via the edges along the perimeter of the delaunay cavity, we need to compute which edge is to the 'left'/'right' of each edge"""
        """don't have the memory to do a batchsize * n * n boolean search, don't have the speed to do a batchsize^2 search (as would be the case for sparse matrix or similar)"""
        """best alternative: rotate the edge around right point, repeat until hit edge in mask (will never go to an edge of a removed triangle before we hit edge in mask) should basically be order 1!"""

        neighbouring_edge_index = neighbouring_edge_mask.nonzero(as_tuple=True)[
            0
        ]  # [?]
        next_neighbouring_edge_index = torch.empty_like(neighbouring_edge_index)  # [?]

        rotating_flipped_neighbouring_edge_index = neighbouring_edge_mask.nonzero(
            as_tuple=True
        )[
            0
        ]  # [?], initialise
        todo_mask = torch.ones_like(
            next_neighbouring_edge_index, dtype=torch.bool
        )  # [?]
        while todo_mask.sum():
            rotating_neighbouring_edge_index = (
                rotating_flipped_neighbouring_edge_index
                + 1
                - 3 * (rotating_flipped_neighbouring_edge_index % 3 == 2)
            )  # [todo_mask.sum()], gets smaller until nothing left EFFICIENCY (this may be seriously stupid, as it requires making a bunch of copies when I could be doing stuff inplace)

            update_mask = neighbouring_edge_mask[
                rotating_neighbouring_edge_index
            ]  # [todo_mask.sum()]
            update_mask_unravel = torch.zeros_like(todo_mask).masked_scatter(
                todo_mask, update_mask
            )  # [?]

            next_neighbouring_edge_index.masked_scatter_(
                update_mask_unravel,
                rotating_neighbouring_edge_index.masked_select(update_mask),
            )  # [?]

            todo_mask.masked_fill_(update_mask_unravel, False)  # [?]
            rotating_flipped_neighbouring_edge_index = edges[
                rotating_neighbouring_edge_index.masked_select(
                    update_mask.logical_not()
                )
            ]  # [todo_mask.sum()]
        triangle_index = new_triangles_mask.view(-1).nonzero(as_tuple=True)[
            0
        ]  # [?], index goes up to batchsize * (ntriangles + 2), this is needed for when we invert the permutation by scattering (won't scatter same number of triangles per batch)

        next_triangle_index = torch.empty_like(edges).masked_scatter_(
            neighbouring_edge_mask, triangle_index
        )[
            next_neighbouring_edge_index
        ]  # [?], index goes up to batchsize * (ntriangles + 2)
        next_edge = 3 * next_triangle_index + 1  # [?]

        invert_permutation = torch.empty_like(
            new_triangles_mask.view(-1), dtype=torch.long
        )  # [batchsize * (ntriangles + 2)]
        invert_permutation[
            next_triangle_index
        ] = triangle_index  # [batchsize * (ntriangles + 2)]
        previous_triangle_index = invert_permutation.masked_select(
            new_triangles_mask.view(-1)
        )  # [?]
        previous_edge = 3 * previous_triangle_index + 2  # [?]

        """in the above we rotated around 'first_point' in our new triangles"""
        new_edge20 = next_edge % ((self.ntriangles + 2) * 3)  # [?]
        new_edge12 = previous_edge % ((self.ntriangles + 2) * 3)  # [?]

        new_edges = torch.stack([new_edge01, new_edge12, new_edge20], dim=1)  # [?, 3]
        self.partial_delaunay_edges.masked_scatter_(
            new_triangles_mask.unsqueeze(2)
            .expand(-1, -1, 3)
            .reshape(self.batchsize, -1),
            new_edges,
        )  # [batchsize, (ntriangles + 2) * 3]

        self.ntriangles += 2
        """currently only count the extra triangles you replace (not the one you have to remove because you're located there, and not the ones you make because you have to create two more"""
        self.cost += n_new_triangles - 3
        self.points_mask.scatter_(
            1, point_index.unsqueeze(1).expand(-1, self.npoints), True
        )
        self.points_sequence = torch.cat(
            [self.points_sequence, point_index.unsqueeze(1)], dim=1
        )


# turn integer x,y coords (in nxn grid) into position d (0 to n^2-1) along the Hilbert curve.
def xy2d(n, x, y):
    [x, y] = [math.floor(x), math.floor(y)]
    [rx, ry, s, d] = [0, 0, 0, 0]
    s = n / 2
    s = math.floor(s)
    while s > 0:
        rx = (x & s) > 0  # bitwise and, and then boolean is it greater than 0?
        ry = (y & s) > 0
        d += s * s * ((3 * rx) ^ ry)
        [x, y] = rot(n, x, y, rx, ry)
        s = s / 2
        s = math.floor(s)
    return d


def rot(n, x, y, rx, ry):
    if ry == 0:
        if rx == 1:
            x = n - 1 - x
            y = n - 1 - y
        # Swap x and y
        t = x
        x = y
        y = t
    return x, y


def order(
    n, points
):  # turns tensor of points into integer distances along hilbert curve of itteration n
    grid = n * points.to("cpu")
    x = torch.empty([grid.size(0)])
    for i in range(points.size(0)):
        x[i] = xy2d(n, grid[i, 0], grid[i, 1])
    return x


def hilbert_insertion(npoints=103, batchsize=200):
    env.reset(npoints, batchsize, nsamples=1)
    points = env.points[:, 3:]  # [batchsize, npoints - 3]
    insertion_order = torch.full([batchsize, npoints], float("inf"), device=device)
    for i in range(batchsize):
        insertion_order[i, 3:] = order(
            2 ** 6, points[i]
        )  # number of possible positions is n ** 2
    for i in range(npoints - 3):
        next_index = insertion_order.min(-1)[1]
        env.update(next_index)
        insertion_order.scatter_(1, next_index.unsqueeze(1), float("inf"))
    print(env.cost.mean().item(), env.cost.var().sqrt().item())


def random_insertion(npoints=103, batchsize=200):
    env.reset(npoints, batchsize, nsamples=1)
    for i in range(npoints - 3):
        env.update(torch.full([batchsize], i + 3, dtype=torch.long, device=device))
    print(env.cost.mean().item(), env.cost.var().sqrt().item())

env = environment()

In [ ]:
for i in range(10):
    random_insertion(1003, 100)

2890.909912109375 42.50613021850586
2891.059814453125 42.404930114746094
2890.579833984375 42.23149490356445
